In [ ]:
# Start writing code here..

In [1]:
import pandas as pd
import sqlite3 # built-in library (Python 2.x & 3.x)

dbpath = "chinook.db" 

conn = sqlite3.connect(dbpath)
cur = conn.cursor() 

# conn.cursor(), conn.commit/rollback(), conn.close() <- conn.commit() : 저장 vs conn.rollback() : 마지막 commit 이후의 변경사항 모두 취소 
# cur.execute/executescript/executemany(), cur.fetchone/fetchall()

script = "SELECT name, milliseconds, albumid FROM tracks;"

df = pd.read_sql_query(script, conn)
df.head()

,Name,Milliseconds,AlbumId
0,For Those About To Rock (We Salute You),343719,1
1,Balls to the Wall,342562,2
2,Fast As a Shark,230619,3
3,Restless and Wild,252051,3
4,Princess of the Dawn,375418,3


In [ ]:
#order by
script = """
SELECT 
    name, 
    milliseconds, 
    albumid 
FROM 
    tracks 
ORDER BY            -- 선택한 특정한 열의 순서(ORDER)에 따라(BY)
    milliseconds;   -- 오름차순 정렬이 디폴트 
"""

df = pd.read_sql_query(script, conn)
df.head()

,Name,Milliseconds,AlbumId
0,É Uma Partida De Futebol,1071,200
1,Now Sports,4884,18
2,A Statistic,6373,18
3,Oprah,6635,18
4,Commercial 1,7941,258


In [ ]:
script = """
SELECT 
    name,
    milliseconds,
    albumid 
FROM 
    tracks 
ORDER BY 
    milliseconds DESC;   -- DESC : 내림차순(Descending) <-> ASC : 오름차순 (Ascending)
""" 

df = pd.read_sql_query(script, conn)
df.head()

,Name,Milliseconds,AlbumId
0,Occupation / Precipice,5286953,227
1,Through a Looking Glass,5088838,229
2,"Greetings from Earth, Pt. 1",2960293,253
3,The Man With Nine Lives,2956998,253
4,"Battlestar Galactica, Pt. 2",2956081,253


In [ ]:
script = """
SELECT
    name,
    milliseconds, 
    albumid
FROM
    tracks
ORDER BY   
    -- 1) albumid 열을 기준으로 오름차순 정렬하고,
    -- 2) 동일한 albumid 값을 가진 행들에 대해서는 milliseconds 열을 기준으로 내림차순 정렬
    albumid ASC,         -- 1순위 조건
    milliseconds DESC;         -- 2순위 조건
""" 

df = pd.read_sql_query(script, conn)
df.head()

,Name,Milliseconds,AlbumId
0,For Those About To Rock (We Salute You),343719,1
1,Spellbound,270863,1
2,Evil Walks,263497,1
3,Breaking The Rules,263288,1
4,Let's Get It Up,233926,1


In [ ]:
script = """
SELECT 
    TrackId, 
    Name, 
    Composer 
FROM 
   tracks
ORDER BY 
   Composer;
""" 

df = pd.read_sql_query(script, conn)
df.head()

# Composer 중 "None"은 Null data, SQLite에서 [ Null data는 가장 작은 값 ]으로 인식 
# 참고 : Sorting NULLs @ https://www.sqlitetutorial.net/sqlite-order-by/

,TrackId,Name,Composer
0,2,Balls to the Wall,None
1,63,Desafinado,None
2,64,Garota De Ipanema,None
3,65,Samba De Uma Nota Só (One Note Samba),None
4,66,Por Causa De Você,None


In [ ]:
#fitering
script = """
SELECT 
    city
FROM 
    customers
ORDER BY 
    city;
""" 

df = pd.read_sql_query(script, conn)
print(len(df))
df.head(10)

59


,City
0,Amsterdam
1,Bangalore
2,Berlin
3,Berlin
4,Bordeaux
5,Boston
6,Brasília
7,Brussels
8,Budapest
9,Buenos Aires


In [ ]:
script = """
SELECT 
    DISTINCT city    -- distinct : 뚜렷이 구별되는, 별개의, 중복되는거 삭제, set()
FROM 
    customers
ORDER BY 
    city;
""" 

df = pd.read_sql_query(script, conn)
print(len(df))
df.head(10)

53


,City
0,Amsterdam
1,Bangalore
2,Berlin
3,Bordeaux
4,Boston
5,Brasília
6,Brussels
7,Budapest
8,Buenos Aires
9,Chicago


In [ ]:
script = """
SELECT 
    DISTINCT city, country    -- city & country [ 2개의 열의 값이 모두 동일한 행 ]들의 경우 중복 제외

FROM
    customers
ORDER BY
    country;
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,City,Country
0,Buenos Aires,Argentina
1,Sidney,Australia
2,Vienne,Austria
3,Brussels,Belgium
4,São José dos Campos,Brazil
5,São Paulo,Brazil
6,Rio de Janeiro,Brazil
7,Brasília,Brazil
8,Montréal,Canada
9,Edmonton,Canada


In [ ]:
script = """
SELECT 
    company
FROM 
    customers;
""" 

df = pd.read_sql_query(script, conn)
print(len(df))
df.head(10)

59


,Company
0,Embraer - Empresa Brasileira de Aeronáutica S.A.
1,None
2,None
3,None
4,JetBrains s.r.o.
5,None
6,None
7,None
8,None
9,Woodstock Discos


In [ ]:
script = """
SELECT 
    DISTINCT company    -- only one row of None(Null) data remains
FROM 
    customers;
""" 

df = pd.read_sql_query(script, conn)
print(len(df))
df.head(10)

11


,Company
0,Embraer - Empresa Brasileira de Aeronáutica S.A.
1,None
2,JetBrains s.r.o.
3,Woodstock Discos
4,Banco do Brasil S.A.
5,Riotur
6,Telus
7,Rogers Canada
8,Google Inc.
9,Microsoft Corporation


- 비교 
=			같다.
<>			같지 않다.
<			작다.
<=			작거나 같다.
\>			크다.
\>=			크거나 같다.  


- 범위
BETWEEN ~ AND ~		~와 ~사이에 있다.
NOT BETWEEN ~ AND ~	~와 ~사이에 없다.  


- 집합
WHERE ~ IN ( , , , ... )	( , , ) 중 하나이다.
WHERE ~ NOT IN (, , , ... )	( , , ) 중 하나도 아니다.  


- 패턴
LIKE ~			~와 비슷하다.
NOT LIKE ~		~와 비슷하지 않다.  


- NULL
IS NULL			NULL이면
IS NOT NULL		NULL이 아니면  


- 복합
~ AND ~			~와 ~에 모두 해당하면
~ OR ~			~혹은 ~에 하나라도 해당하면
NOT ~			~에 해당하지 않으면  


- 문자열 패턴
%wild%          wild를 포함하는가
[0-9]%          숫자 0\~9 중 하나로 시작하는가
[^0-9]%         숫자 0\~9 중 하나로 시작하지 않는가
%Br_wn          Br과 wn사이에 문자열이 1개 존재하는가
'hello'+'world' hello와 world 연결하기 -> 'hello world'

In [2]:
# WHERE Examples

# WHERE column_1 = 100;
# WHERE column_2 IN (1,2,3);
# WHERE column_3 LIKE 'An%';
# WHERE column_4 BETWEEN 10 AND 20;

In [3]:
script = """
SELECT
   name,
   milliseconds,
   bytes,
   albumid
FROM              -- FROM 테이블명
   tracks
WHERE             -- WHERE 조건절 (테이블을 선택하는 FROM 과 헷갈리지 않도록 유의합니다.)
   albumid = 10;  -- <> 같지 않은 것들 
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,Name,Milliseconds,Bytes,AlbumId
0,Cochise,222380,5339931,10
1,Show Me How to Live,277890,6672176,10
2,Gasoline,279457,6709793,10
3,What You Are,249391,5988186,10
4,Like a Stone,294034,7059624,10
5,Set It Off,263262,6321091,10
6,Shadow on the Sun,343457,8245793,10
7,I am the Highway,334942,8041411,10
8,Exploder,206053,4948095,10
9,Hypnotize,206628,4961887,10


In [4]:
script = """
SELECT
   name,
   milliseconds,
   bytes,
   albumid
FROM
   tracks
WHERE
   (albumid = 10) AND (milliseconds > 250000);   -- 2가지 조건을 모두(AND) 만족시키는 행만 선택하려면?
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,Name,Milliseconds,Bytes,AlbumId
0,Show Me How to Live,277890,6672176,10
1,Gasoline,279457,6709793,10
2,Like a Stone,294034,7059624,10
3,Set It Off,263262,6321091,10
4,Shadow on the Sun,343457,8245793,10
5,I am the Highway,334942,8041411,10
6,Bring'em Back Alive,329534,7911634,10
7,Light My Way,303595,7289084,10
8,Getaway Car,299598,7193162,10
9,The Last Remaining Light,317492,7622615,10


In [6]:
script = """
SELECT
    TrackId,
    Name,
    MediaTypeId
FROM
    Tracks
WHERE
    (MediaTypeId = 1) OR (MediaTypeId = 2);    -- 2가지 조건 중 하나라도(OR) 만족시키는 행만 선택하려면?
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,TrackId,Name,MediaTypeId
0,1,For Those About To Rock (We Salute You),1
1,6,Put The Finger On You,1
2,7,Let's Get It Up,1
3,8,Inject The Venom,1
4,9,Snowballed,1
5,10,Evil Walks,1
6,11,C.O.D.,1
7,12,Breaking The Rules,1
8,13,Night Of The Long Knives,1
9,14,Spellbound,1


In [7]:
script = """
SELECT
    trackid,
    name
FROM
    tracks
WHERE
    name LIKE 'Wild%';   -- name 열의 값이 [ "Wild" 로 시작하고 ] & [ 뒤에 0개 이상의 문자열이 있는(%) ] 행들을 선택
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,TrackId,Name
0,1245,Wildest Dreams
1,1973,Wild Side
2,2627,Wild Hearted Son
3,2633,Wild Flower
4,2944,Wild Honey


In [8]:
script = """
SELECT
    trackid,
    name
FROM
    tracks
WHERE
    name LIKE '%Wild';    -- name 열의 값이 [ "Wild" 로 끝나고 ] & [ 앞에 0개 이상의 문자열이 있는(%) ] 행들을 선택
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,TrackId,Name
0,4,Restless and Wild
1,32,Deuces Are Wild
2,775,Call Of The Wild
3,2697,I Go Wild


In [9]:
script = """
SELECT
    trackid,
    name
FROM
    tracks
WHERE
    name LIKE '%Wild%';    -- [ "Wild" 가 사이에 들어 있고 ] & [ 앞 혹은 뒤에 0개 이상의 문자열이 있는(%) ] 행들을 선택
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,TrackId,Name
0,4,Restless and Wild
1,32,Deuces Are Wild
2,775,Call Of The Wild
3,1245,Wildest Dreams
4,1869,Where The Wild Things Are
5,1973,Wild Side
6,2312,Near Wild Heaven
7,2627,Wild Hearted Son
8,2633,Wild Flower
9,2697,I Go Wild


In [11]:
script = """
SELECT
    trackid,
    name
FROM
    tracks
WHERE
    name LIKE '%Br_wn%';    -- "_" : 해당 위치의 문자 1개와 매칭됨 (무엇이든 문자 1개가 "_" 자리에 위치해있으면 ok)
                            -- ex) Brown, Brywn, Brawn, Br1wn, etc.
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,TrackId,Name
0,1053,"Bad, Bad Leroy Brown"
1,1150,Mr. Brownstone


In [13]:
script = """
SELECT
    name,
    albumid,
    mediatypeid
FROM
    tracks
WHERE
    mediatypeid in (4, 5);      -- mediatypeid 열의 값이 4 혹은 5 중에 속하는(IN) 행들을 선택 
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,Name,AlbumId,MediaTypeId
0,War Pigs,260,4
1,"Symphony No. 104 in D Major ""London"": IV. Fina...",283,4
2,"SCRIABIN: Prelude in B Major, Op. 11, No. 11",318,4
3,"Prometheus Overture, Op. 43",324,4
4,Sonata for Solo Violin: IV: Presto,325,4
5,"Étude 1, In C Major - Preludio (Presto) - Liszt",340,4
6,"Concerto for Violin, Strings and Continuo in G...",342,4
7,Amanda,262,5
8,Despertar,262,5
9,Din Din Wo (Little Child),263,5


In [14]:
script = """
SELECT
    name,
    albumid,
    mediatypeid
FROM
    tracks
WHERE
    mediatypeid NOT IN (4, 5);     -- mediatypeid 열의 값이 4 혹은 5 중에 속하지(IN) 않는(NOT) 행들을 선택 
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,Name,AlbumId,MediaTypeId
0,For Those About To Rock (We Salute You),1,1
1,Balls to the Wall,2,2
2,Fast As a Shark,3,2
3,Restless and Wild,3,2
4,Princess of the Dawn,3,2
5,Put The Finger On You,1,1
6,Let's Get It Up,1,1
7,Inject The Venom,1,1
8,Snowballed,1,1
9,Evil Walks,1,1


In [17]:
#sub query
script = """
SELECT
    TrackId, 
    Name, 
    AlbumId
FROM
    Tracks
WHERE
    AlbumId IN (      -- Albums 테이블에서 ArtistId 열의 값이 12인 행들의 AlbumId 열의 값들 <- 이 AlbumId 값들이 타겟 그룹이 됨
        SELECT
            AlbumId
        FROM
            Albums
        WHERE
            ArtistID = 12
    );
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,TrackId,Name,AlbumId
0,149,Black Sabbath,16
1,150,The Wizard,16
2,151,Behind The Wall Of Sleep,16
3,152,N.I.B.,16
4,153,Evil Woman,16
5,154,Sleeping Village,16
6,155,Warning,16
7,156,Wheels Of Confusion / The Straightener,17
8,157,Tomorrow's Dream,17
9,158,Changes,17


In [18]:
script = """
SELECT
    trackId,
    name
FROM
    tracks
LIMIT 4;    -- 제한(LIMIT)을 걸어 위에서부터 차례대로 4개까지만 데이터를 읽어들인다.
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,TrackId,Name
0,1,For Those About To Rock (We Salute You)
1,2,Balls to the Wall
2,3,Fast As a Shark
3,4,Restless and Wild


In [19]:
script = """
SELECT
    trackId,
    name
FROM
    tracks
LIMIT 10 OFFSET 7; -- OFFSET의 값에 해당하는 수만큼 떼어(OFF)내고(SET) 그 이후의 데이터부터 가져온다.
                   -- 적용 순서에 유의 (먼저 OFFSET으로 먼저 떼어낸 다음, LIMIT 제한 수만큼 데이터를 읽어들임)
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,TrackId,Name
0,8,Inject The Venom
1,9,Snowballed
2,10,Evil Walks
3,11,C.O.D.
4,12,Breaking The Rules
5,13,Night Of The Long Knives
6,14,Spellbound
7,15,Go Down
8,16,Dog Eat Dog
9,17,Let There Be Rock


In [20]:
script = """
SELECT
    trackid,
    name,
    milliseconds
FROM
    tracks
ORDER BY
    milliseconds ASC
LIMIT 10;              -- LIMIT & OFFSET은 SELECT 관련 키워드 중 가장 마지막에 위치함
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,TrackId,Name,Milliseconds
0,2461,É Uma Partida De Futebol,1071
1,168,Now Sports,4884
2,170,A Statistic,6373
3,178,Oprah,6635
4,3304,Commercial 1,7941
5,172,The Real Problem,11650
6,3310,Commercial 2,21211
7,2241,Bossa,29048
8,1086,Casinha Feliz,32287
9,246,Mateus Enter,33149


In [21]:
script = """
SELECT
    trackid,
    name,
    milliseconds
FROM
    tracks
ORDER BY
    milliseconds DESC    
LIMIT 1 OFFSET 1;        -- milliseconds(해당 곡의 길이, 1000ms=1s) 기준으로 2번째로 가장 긴 길이를 갖고 있는 곡을 확인하려면?
                         -- 적용 순서에 유의 (먼저 OFFSET으로 먼저 떼어낸 다음, LIMIT 제한 수만큼 데이터를 읽어들임)
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

# the second-longest track in the tracks table.

,TrackId,Name,Milliseconds
0,3224,Through a Looking Glass,5088838


In [22]:
script = """
SELECT
    InvoiceId,
    BillingAddress,
    Total
FROM
    invoices
WHERE
    Total BETWEEN 14.91 AND 18.86    -- 전체 주문 수량(Total)의 값이 14.91과 18.86 사이에 해당하는 행 (between A and B)
ORDER BY
    Total; 
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,InvoiceId,BillingAddress,Total
0,193,Berger Straße 10,14.91
1,103,162 E Superior Street,15.86
2,208,Ullevålsveien 14,15.86
3,306,Klanova 9/506,16.86
4,313,"68, Rue Jouvence",16.86
5,88,"Calle Lira, 198",17.91
6,89,"Rotenturmstraße 4, 1010 Innere Stadt",18.86
7,201,319 N. Frances Street,18.86


In [23]:
script = """
SELECT
    InvoiceId,
    BillingAddress,
    Total
FROM
    invoices
WHERE
    Total NOT BETWEEN 1 AND 20     -- 전체 주문 수량(Total)의 값이 [1과 20 사이에 해당하지 않는(NOT)] 행 
ORDER BY
    Total;
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,InvoiceId,BillingAddress,Total
0,6,Berger Straße 10,0.99
1,13,1600 Amphitheatre Parkway,0.99
2,20,110 Raeburn Pl,0.99
3,27,5112 48 Street,0.99
4,34,"Praça Pio X, 119",0.99
5,41,C/ San Bernardo 85,0.99
6,48,796 Dundas Street West,0.99
7,55,Grétrystraat 63,0.99
8,62,3 Chatham Street,0.99
9,69,319 N. Frances Street,0.99


In [24]:
script = """
SELECT
    InvoiceId,
    BillingAddress,
    InvoiceDate,
    Total
FROM
    invoices
WHERE
    InvoiceDate BETWEEN '2010-01-01' AND '2010-01-31'
ORDER BY
    InvoiceDate;    
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

,InvoiceId,BillingAddress,InvoiceDate,Total
0,84,"68, Rue Jouvence",2010-01-08 00:00:00,1.98
1,85,Erzsébet krt. 58.,2010-01-08 00:00:00,1.98
2,86,"Via Degli Scipioni, 43",2010-01-09 00:00:00,3.96
3,87,Celsiusg. 9,2010-01-10 00:00:00,6.94
4,88,"Calle Lira, 198",2010-01-13 00:00:00,17.91
5,89,"Rotenturmstraße 4, 1010 Innere Stadt",2010-01-18 00:00:00,18.86
6,90,801 W 4th Street,2010-01-26 00:00:00,0.99


In [3]:
script = """
SELECT
    Name, 
    Composer
FROM
    tracks
WHERE
    Composer IS NULL
ORDER BY 
    Name; 
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

NameError: name 'pd' is not defined

In [2]:
script = """
SELECT
    Name, 
    Composer
FROM
    tracks
WHERE
    Composer IS NOT NULL
ORDER BY 
    Name; 
""" 

df = pd.read_sql_query(script, conn)
df.head(10)

NameError: name 'pd' is not defined

In [27]:
conn.close()

- AnB
InnerJoin		A와 B의 교집합
Left		집합 A로 Join
Right		집합 B로 Join
Outer		A와 B의 합집합


- Inner Join
SELECT ~ From talbe_a INNER JOIN table_b ON table_a.f = table_b.f
SELECT albums.title, artists.name FROM albums INNER JOIN artists ON artists.ArtistId = albums.ArtistId;
SELECT
    l.Title, 
    r.Name    -- 아래 테이블 이름 바로 다음에 alias를 지정해주면 l. & r. 와 같이 alias를 활용할 수 있습니다.
FROM
    albums l 
INNER JOIN 
    artists r 
ON 
    r.ArtistId = l.ArtistId;


SELECT
   Title, 
   Name
FROM
   albums
INNER JOIN artists ?(ArtistId);   -- key가 되는 열의 이름(여기서는 'ArtistId')이 같을 경우, 
                                      -- [ USING(열이름) ] 으로 [ A.열이름 = B.열이름 ]을 대체할 수 있습니다.



- SELF JOIN

SELECT m.firstname || ' ' || m.lastname AS 'Manager',
       e.firstname || ' ' || e.lastname AS 'Receives reports from'  
FROM 
    employees e
INNER JOIN 
    employees m 
ON 
    m.employeeid = e.reportsto    -- 테이블 employees의 복제본이 존재한다고 상상하고 2개의 동일한 테이블을 서로 JOIN 한다고 이해
ORDER BY 
    manager;



In [1]:
import pandas as pd
import sqlite3 # built-in library (Python 2.x & 3.x)

dbpath = "chinook.db" 

conn = sqlite3.connect(dbpath)
cur = conn.cursor() 

# conn.cursor(), conn.commit/rollback(), conn.close() <- conn.commit() : 저장 vs conn.rollback() : 마지막 commit 이후의 변경사항 모두 취소 
# cur.execute/executescript/executemany(), cur.fetchone/fetchall()

script = "SELECT name, milliseconds, albumid FROM tracks;"

df = pd.read_sql_query(script, conn)
df.head()

,Name,Milliseconds,AlbumId
0,For Those About To Rock (We Salute You),343719,1
1,Balls to the Wall,342562,2
2,Fast As a Shark,230619,3
3,Restless and Wild,252051,3
4,Princess of the Dawn,375418,3


In [2]:
script = """
SELECT 
    *
FROM 
    albums
""" 
cur.execute(script) 

result = cur.fetchall()
result[0:5]

[(1, 'For Those About To Rock We Salute You', 1),
 (2, 'Balls to the Wall', 2),
 (3, 'Restless and Wild', 2),
 (4, 'Let There Be Rock', 1),
 (5, 'Big Ones', 3)]

In [3]:
script = """
SELECT 
    *
FROM 
    artists
""" 

df = pd.read_sql_query(script, conn)
df.head()

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [4]:
script = """
SELECT 
    Title,    -- <- albums(A)
    Name      -- <- artists(B)
FROM 
    albums    -- Table albums(A)
INNER JOIN 
    artists   -- Table artists(B)
ON 
    artists.ArtistId = albums.ArtistId;   -- 순서는 바뀌어도 괜찮습니다.
"""

df = pd.read_sql_query(script, conn)
df.head()

,Title,Name
0,For Those About To Rock We Salute You,AC/DC
1,Balls to the Wall,Accept
2,Restless and Wild,Accept
3,Let There Be Rock,AC/DC
4,Big Ones,Aerosmith


In [7]:
script = "SELECT * FROM albums"
df = pd.read_sql_query(script, conn)
df.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [9]:
script = """
SELECT 
    albums.Title,
    artists.Name    -- 이처럼 Table명을 밝혀서 써주면 더욱 코드가 명료해집니다.
FROM 
    albums
INNER JOIN
    artists
ON 
    artists.ArtistId = albums.ArtistId;    -- from A inner join B on A.x1 = B.x2
"""

df = pd.read_sql_query(script, conn)
df.head()

,Title,Name
0,For Those About To Rock We Salute You,AC/DC
1,Balls to the Wall,Accept
2,Restless and Wild,Accept
3,Let There Be Rock,AC/DC
4,Big Ones,Aerosmith


In [10]:
script = """
-- 특정한 곡(tracks)에 대하여 해당 곡이 포함된 앨범(albums)과 해당 앨범을 제작한 아티스트(artists)의 정보를 함께 확인하고자 합니다.
-- tracks : 특정한 곡의 정보 (album 정보 포함)
-- albums : 여러 곡들을 포함하고 있는 특정한 앨범의 정보 (tracks & artists 정보 포함)
-- artists : 특정한 앨범에 대한 아티스트의 정보 (album 정보 포함)

SELECT
    trackid,
    tracks.name AS track,   -- AS 를 통해 alias를 만들어줄 수 있습니다. (아래 실행 결과를 참고)
    albums.title AS album,
    artists.name AS artist
FROM
    tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
INNER JOIN artists ON artists.artistid = albums.artistid;
""" 

df = pd.read_sql_query(script, conn)
df.head()

,TrackId,track,album,artist
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC
1,6,Put The Finger On You,For Those About To Rock We Salute You,AC/DC
2,7,Let's Get It Up,For Those About To Rock We Salute You,AC/DC
3,8,Inject The Venom,For Those About To Rock We Salute You,AC/DC
4,9,Snowballed,For Those About To Rock We Salute You,AC/DC


In [11]:
script = """
SELECT
    trackid,
    tracks.name AS Track,
    albums.title AS Album,
    artists.name AS Artist,
    artists.artistid AS ArtistID  -- 이전 실행 결과에 새로이 추가됨
FROM
    tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
INNER JOIN artists ON artists.artistid = albums.artistid
WHERE
    artists.artistid = 10;  -- 이전 실행 결과에 새로이 추가됨 
                            -- (JOIN 대상이 되는 table 중 원하는 table을 기준으로 조건을 정할 수 있습니다.)
""" 

df = pd.read_sql_query(script, conn)
df.head()

,TrackId,Track,Album,Artist,ArtistID
0,123,Quadrant,The Best Of Billy Cobham,Billy Cobham,10
1,124,Snoopy's search-Red baron,The Best Of Billy Cobham,Billy Cobham,10
2,125,"Spanish moss-""A sound portrait""-Spanish moss",The Best Of Billy Cobham,Billy Cobham,10
3,126,Moon germs,The Best Of Billy Cobham,Billy Cobham,10
4,127,Stratus,The Best Of Billy Cobham,Billy Cobham,10


In [12]:
script = """
SELECT
    Name, 
    Title
FROM
    artists
LEFT JOIN albums ON artists.ArtistId = albums.ArtistId
ORDER BY 
    Name;
""" 

df = pd.read_sql_query(script, conn)
df.head(15) 

# There are multiple 'None'(Null)s in the right column

,Name,Title
0,A Cor Do Som,None
1,AC/DC,For Those About To Rock We Salute You
2,AC/DC,Let There Be Rock
3,Aaron Copland & London Symphony Orchestra,"A Copland Celebration, Vol. I"
4,Aaron Goldberg,Worlds
5,Academy of St. Martin in the Fields & Sir Nevi...,The World of Classical Favourites
6,Academy of St. Martin in the Fields Chamber En...,Sir Neville Marriner: A Celebration
7,"Academy of St. Martin in the Fields, John Birc...","Fauré: Requiem, Ravel: Pavane & Others"
8,"Academy of St. Martin in the Fields, Sir Nevil...",Bach: Orchestral Suites Nos. 1 - 4
9,"Academy of St. Martin in the Fields, Sir Nevil...",None


In [13]:
script = """
SELECT
    Name,  -- <- artists (A)
    Title  -- <- albums (B)
FROM
    artists
LEFT JOIN albums ON artists.ArtistId = albums.ArtistId
WHERE Title IS NULL
ORDER BY Name;
""" 

df = pd.read_sql_query(script, conn)
df.head(15) 

,Name,Title
0,A Cor Do Som,None
1,"Academy of St. Martin in the Fields, Sir Nevil...",None
2,Aerosmith & Sierra Leone's Refugee Allstars,None
3,Avril Lavigne,None
4,Azymuth,None
5,Baby Consuelo,None
6,Banda Black Rio,None
7,Barão Vermelho,None
8,Bebel Gilberto,None
9,Ben Harper,None


In [14]:
script = """
SELECT *
FROM 
    employees
LIMIT 3;
""" 

df = pd.read_sql_query(script, conn)
df.head() 

# 아래 실행 결과에서 ReportsTo 열의 숫자는 [해당 직원이 보고해야할 상급자의 EmployeeId 값]입니다.
# 누가 누구로부터 보고를 받는지 확인하고자 합니다.

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com


In [15]:
#selfjoin
script = """
SELECT m.firstname || ' ' || m.lastname AS 'Manager',
       e.firstname || ' ' || e.lastname AS 'Receives reports from'  
FROM 
    employees e
INNER JOIN 
    employees m 
ON 
    m.employeeid = e.reportsto    -- 테이블 employees의 복제본이 존재한다고 상상하고 2개의 동일한 테이블을 서로 JOIN 한다고 이해
ORDER BY 
    manager;
""" 

df = pd.read_sql_query(script, conn)
df.head() 

,Manager,Receives reports from
0,Andrew Adams,Nancy Edwards
1,Andrew Adams,Michael Mitchell
2,Michael Mitchell,Robert King
3,Michael Mitchell,Laura Callahan
4,Nancy Edwards,Jane Peacock


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4d5c3f66-c043-43c9-ab39-85c790f02501' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>